In [1]:
from datasets import load_dataset
from openicl import DatasetReader


# Loading dataset from huggingface
dataset = load_dataset('/data/lhb/huggingface/dataset/sst2_gpt3mix')

# Define a DatasetReader, with specified column names where input and output are stored.
data = DatasetReader(dataset, input_columns=['text'], output_column='label')

In [2]:
from openicl import PromptTemplate
tp_dict = {
    0: "</E>Positive Movie Review: </text>",
    1: "</E>Negative Movie Review: </text>" 
}

template = PromptTemplate(tp_dict, {'text': '</text>'}, ice_token='</E>')

In [3]:
from openicl import TopkRetriever
from openicl.icl_retriever.icl_cone_retriever import ConERetriever
# 这一步会将 训练集中的 所有数据 进行 embedding 在内存中 构建一个 faiss 索引，只有在 检索的时候 才会对所有 测试集中的 数据 进行 embedding 和 建立索引

retriever = TopkRetriever(data, ice_num=8)
# retriever = ConERetriever(data, candidate_num = 30, ice_num=8)

[2025-03-09 20:15:28,376] [openicl.icl_retriever.icl_topk_retriever] [INFO] Creating index for index set...
100%|██████████| 6920/6920 [01:47<00:00, 64.54it/s]


In [4]:
from openicl import PPLInferencer
inferencer = PPLInferencer(model_name='/data/lhb/huggingface/model/tokenizer/gpt2-xl', output_json_filepath = "./icl_inference_output", output_json_filename = "predictions")

In [5]:
from openicl import AccEvaluator
# the inferencer requires retriever to collect in-context examples, as well as a template to wrap up these examples.
predictions = inferencer.inference(retriever, ice_template=template)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


AttributeError: 'NoneType' object has no attribute 'template'

In [7]:
# compute accuracy for the prediction
score = AccEvaluator().score(predictions=predictions, references=data.references)
print(score)

{'accuracy': 0.8396485447556288}


In [7]:
len(predictions)

1821

In [8]:
len(data.references)

1821

In [16]:
import evaluate

def score(predictions, references):
    assert len(predictions) == len(references)
    mapping_to_int_dict = {label: idx for idx, label in enumerate(set(map(str, references)))}
    pred_set = set(predictions)
    for pred in pred_set:
        if str(pred) not in mapping_to_int_dict.keys():
            mapping_to_int_dict[str(pred)] = len(mapping_to_int_dict)
    golds = [mapping_to_int_dict[str(gold)] for gold in references]
    preds = [mapping_to_int_dict[str(pred)] for pred in predictions]
    metric = evaluate.load("/data/lhb/huggingface/metrics/accuracy")
    return metric.compute(references=golds, predictions=preds)

In [17]:
score(references=data.references, predictions=predictions)

{'accuracy': 0.8396485447556288}

In [ ]:
retriever

### 将 topk_search 函数拆分调试

In [7]:
import numpy as np
# 测试集前向传播，得到 {'embed'...，'metadata':..}
np.random.seed(42)
res_list = retriever.forward(retriever.dataloader)
rtr_idx_list = [[] for _ in range(len(res_list))]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
key_word = retriever.ice_template.template[0].split('</text>')[-1].split()[0]

AttributeError: 'NoneType' object has no attribute 'template'

In [25]:
retriever.ice_template.template[0].split('</text>')[-1].split()[0]

IndexError: list index out of range